In [30]:
!pip install langchain-community
!pip install pypdf

In [31]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [32]:
pdf_files = ["pdf/cristiano.pdf","pdf/messi.pdf","pdf/neymar.pdf"]  
all_documents = []
for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Divide el texto en chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

In [33]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/bda/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdfs",
    embedding=embeddings, 
)

In [35]:
retriever = vector_store.as_retriever()

In [36]:
question = "¿Qué son estos tres jugadores de fútbol?"
docs = vector_store.similarity_search(question, k=10)

In [37]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM: Se configura el modelo llama3.2 para generar respuestas de manera local
llm = "llama3.2"
modelo = ChatOllama(model=llm)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | modelo
    | StrOutputParser()
)

In [38]:
chain.invoke("Quien es messi?")

'Lionel Andrés Messi Cuccittini, conocido como Leo Messi, es un futbolista argentino que juega como delantero o centrocampista. Es capitán de la selección de Argentina y ha jugado en el Fútbol Club Barcelona durante más de veinte años, con quien ha ganado numerosos premios y récords, incluyendo cuatro Botas de Oro del Balón de Oro.'

In [39]:
chain.invoke("Quien es cristiano Ronaldo?")

'Cristiano Ronaldo dos Santos Aveiro es un futbolista portugués. Nació en Funchal, Madeira, el 5 de febrero de 1985. Es delantero y actualmente juega para Al-Nassr F. C. de la Liga Profesional Saudí. Además, es capitán de la selección de Portugal, máximo goleador histórico y jugador con más presencias en la historia de la selección con 217 partidos.'

In [40]:
chain.invoke("¿Quien es Neymar Jr?")

'Neymar da Silva Santos Júnior, también conocido como Neymar Jr., es un futbolista brasileño que juega como delantero en el Al-Hilal Saudi F.C. de la Liga Profesional Saudí y es internacional absoluto con la selección de Brasil.'